In [ ]:
# Install all requirements from PyPI.
!python3 -m pip install numpy scipy matplotlib numba tensorflow-gpu librosa sndfile mir_eval soundfile requests flask frozendict --user

In [ ]:
# Let's get down to business.
# We need to start a evolution server to manage our evolution process.
# This server needs no GPU or fast CPU, it runs like a charm on a very low-end server.
# Enter the `srv` directory
%cd srv

# You should edit `config.py`, modify these variables properly:
# listen_addr, listen_port, n_gen, evo_type, worker_type
# For local network, set listen_addr to your local ip address.
# For Internet, install nginx, setup reverse proxy and http authentication.
# ⚠️SECURITY WARNING: Do NOT expose your server directly to public network. The server has no security check.⚠️
# Then run it.
!python3 main_srv.py
# You can exit the server by Ctrl-C and restart it at any time, the last unfinished checkpoint will loss.
# When all tasks are finished, the server will exit automatically.

# Notice: To reproduce the paper, you need run the server multiple times with conifguration below:
# n_gen=50, evo_type="g1", worker_type="mir2"
# n_gen=50, evo_type="g1", worker_type="dsd2"
# n_gen=100, evo_type="nsga2", worker_type="mir2"
# n_gen=100, evo_type="nsga2", worker_type="dsd2"

# Return to the project root directory
%cd ..

In [ ]:
# Okay, now we have evolution checkpoints.
# To reproduce our paper, ensure checkpoint directory below has been generated:
# `v1_g1_mir2_0.0` `v1_g1_dsd2_0.0` `v1_nsga2_mir2_0.0` `v1_nsga2_dsd2_0.0`
# Let's do a full training and evaluating for structures that we interested in.
# There are too many gene structures to train and evaluate, so we have to do these work automatically.
# We used the `eval_script.gen` file to manage all gene structures.
# Enter the `final_eval` directory
%cd final_eval

# Generate `eval_script.gen` from evolution checkpoints automatically.
# The `plot_g1_stat.py` reads Single Objective evolution checkpoints and print best gene over generations.
# The `plot_nsga2_stat.py` reads NSGA2 evolution checkpoints and print below genes over generations:
## The gene which has least paramter count within a generation
## The gene which has near-25% highest parameter count(rec_25 line in figure)
## The gene which has near-33% highest parameter count
## The gene which has near-50% highest parameter count(rec_50 line in figure)
## The gene which has near-66% highest parameter count
## The gene which has near-75% highest parameter count(rec_75 line in figure)
## The gene which has near-90% highest parameter count
## The gene which has best score within a generation
# Syntax of `eval_script.gen`:
# {str:ver} {str:g1|nsga2} {str:mir2|dsd2} {int:gene} {str:internal_name} [str:external_name]
# ! mark means a commented line

# Load the module `../lib/redirect.py`
import sys
sys.path.append("../lib")
import redirect
%matplotlib inline

# Redirect stdout to StringIO buffer to capture outputs from `plot_g1_stat.py` and `plot_nsga2_stat.py`
# The plotted figure
with redirect.ConsoleToBuffer(redirect_stdout=True, redirect_stderr=False) as r:
    print("* mir2", file=r.real_stdout)
    print("!!! MIR2")
    print("v1 any mir2 4182591019167972528534244115322478782824676 manual Seed") # Output the seed gene manually
    %run ../figtex/plot_g1_stat.py -dataset=mir2 -score=test -input=../srv/v1_g1_mir2_0.0
    %run ../figtex/plot_nsga2_stat.py -dataset=mir2 -score=test -input=../srv/v1_nsga2_mir2_0.0

    print("* dsd2", file=r.real_stdout)
    print("\n!!! DSD2")
    print("v1 any dsd2 4182591019167972528534244115322478782824676 manual Seed") # Output the seed gene manually
    %run ../figtex/plot_g1_stat.py -dataset=dsd2 -score=test -input=../srv/v1_g1_dsd2_0.0
    %run ../figtex/plot_nsga2_stat.py -dataset=dsd2 -score=test -input=../srv/v1_nsga2_dsd2_0.0
    txt = r.fake_stdout.getvalue()
print(txt)

# Write `eval_script.gen`
with open("eval_script.gen", "wb") as f:
    f.write(txt.encode("utf-8"))

In [ ]:
# We got the `eval_script.gen`, let's generate a bash script to start the training and evaluating process.
# Notice that we still in the `final_eval` directory

# Because there are too many gene structures, it would be better to run these tasks in multiple computers.
# The script `gen_eval.py` can estimate a very coarse speed for several GPU model, and assign task 
# to each machines with balance.
# For example run `python3 gen_eval.py titan_rtx@pc titan_v@mattia titan_xp@pluto 1080ti@judas 1080ti@deepin`
# will generate scripts for these 5 machines with specified GPU model.
#
# In this jupyter notebook, as a demo, we generate script for only single machine.
!python3 gen_eval.py gpumodel@hostname # generate `do_hostname.sh`

# Run the generated bash script.
# It will take a really long time if you have only one GPU.
# Modify variables in `config.py`, set correct dataset path before run this bash script!
!bash -e do_hostname.sh

# Once you are finished, checkpoints and evaluating results will be stored in 
# directory `./ckpt` and `./eval_output`. Copy and merge them from your machines.

# Filter evalutaed data into directory `./extracted_eval_result`
!python3 extract_eval_result.py -step63000 -step630000 # by default 63000 for mir2, 630000 for dsd2

In [ ]:
# Let's generate figures and LaTeX files from our experimental results
# Enter the `figtex` directory
%cd ../figtex/

# Draw all figures and tables
!python3 gen_net_performance_tab.py
!bash -e draw_all.sh

In [ ]:
# Unmix demo songs and draw spectrogram used in paper
# Modify `split_over_all.sh`, set `DATASET_PATH` to your MIR-1K Wavefile path.
# Modify `gene_list.sh`, set correct `GENE_LIST` and `NAME_LIST`.
!bash -e split_over_all.sh geniusturtle_5_03
!bash -e split_over_all.sh geniusturtle_6_04
# Output `../analyze_*`

# Return to the project root directory
%cd ..

In [ ]:
# We have done all the stuff in our paper。
# You can use trained models with `split_song.py`
# Here is an example.
# Enter the `final_eval` directory
%cd final_eval

# Find a gene value from `eval_script.gen`, fill into the argument
# `-no_mix` means we don't want to mixdown the input file into mono
!python3 split_song.py -ver=v1 -dataset=<str> -gene=<int> -step=<int> -no_mix -input=<wavfile>
# produces `split_inst_<filename>.wav` and `split_vocal_<filename>.wav`
# For example, !python3 split_song.py -ver=v1 -dataset=mir2 -gene=4182591019167972528534244115322478782824676 -step=63000 -no_mix -input=./one_half.wav
# will produce `split_inst_one_half.wav` and `split_vocal_one_half.wav`.